In [1]:
versioninfo()
println()

function f(x, T=Float64)
    n = 1
    s = one(T)
    while s < x
        n += 1
        s += inv(T(n))
    end
    n, s
end

@time f(21)

Julia Version 1.7.0-DEV.1133
Commit db8d09609c (2021-05-21 13:48 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)
Environment:
  JULIA_DEPOT_PATH = D:\.julia
  JULIA_NUM_THREADS = 12
  JULIA_PYTHONCALL_EXE = D:\.julia\conda\3\python.exe

  0.651148 seconds


(740461601, 21.000000000402316)

In [2]:
using BenchmarkHistograms
@benchmark f(21)

samples: 8; evals/sample: 1; memory estimate: 0 bytes; allocs estimate: 0
ns

 (6.44e8 - 6.49e8]  ██████████████████████████████ 3
 (6.49e8 - 6.54e8]  ████████████████████ 2
 (6.54e8 - 6.6e8 ]   0
 (6.6e8  - 6.65e8]  ██████████████████████████████ 3

                  Counts

min: 643.559 ms (0.00% GC); mean: 653.062 ms (0.00% GC); median: 651.152 ms (0.00% GC); max: 665.194 ms (0.00% GC).

In [3]:
run(`gcc --version`)
flush(stdout)

C_code = raw"""
long long f(double x) {
    long long n = 1;
    double s = 1.0;
    while (s < x) {
        n++;
        s += 1.0 / (double) n;
    }
    return n;
}
"""
display("text/markdown", "```C\n"*C_code*"\n```")

using Libdl
libname = tempname()
libname_dl = libname * "." * Libdl.dlext

open(`gcc -Wall -O3 -march=native -xc -shared -o $libname_dl -`, "w") do f
     print(f, C_code)
end
run(`ls -l $libname_dl`)
println()

f_gcc(x::Float64) = @ccall libname.f(x::Float64)::Int64
@time f_gcc(21.0)

gcc (Rev10, Built by MSYS2 project) 10.2.0
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



```C
long long f(double x) {
    long long n = 1;
    double s = 1.0;
    while (s < x) {
        n++;
        s += 1.0 / (double) n;
    }
    return n;
}

```

-rwxr-xr-x 1 genkuroki genkuroki 95812 Jun  6 09:06 C:\Users\GENKUR~1\AppData\Local\Temp\jl_VteFNoXkzQ.dll

  0.674273 seconds


740461601

In [4]:
@benchmark 6000125006293f_gcc(21.0)

samples: 8; evals/sample: 1; memory estimate: 0 bytes; allocs estimate: 0
ns

 (6.41e8 - 6.52e8]  ██████████████████████████████ 5
 (6.52e8 - 6.64e8]  ██████ 1
 (6.64e8 - 6.75e8]  ██████ 1
 (6.75e8 - 6.87e8]  ██████ 1

                  Counts

min: 640.577 ms (0.00% GC); mean: 656.097 ms (0.00% GC); median: 649.936 ms (0.00% GC); max: 687.046 ms (0.00% GC).

In [5]:
0.67/740461601*6000125006293/60^2

1.5080997180929478